### Creating function which are needed.

- Function to find all which meet condition, first and second but also specifiy the place.
- Some form of way to print the frequecy of each. This could just use the itertools.

In [2]:
from methods import LMEO, Static
lm = LMEO()

In [3]:
# Filter function.
def filter_perm_in_pos(all_ranks, first_hotel_id, second_hotel_id, pos0 = 0):
    '''This filters by a combo i, j or j, i starting at pos0'''
    i, j = pos0, pos0 + 1
    
    def create_condition(hotel_id_obj, place):
        if isinstance(hotel_id_obj, list):
            return lambda rank: any([hotel_id in rank[place] for hotel_id in hotel_id_obj])
        else:
            return lambda rank: hotel_id_obj in rank[place]
    
    first_in_zero = create_condition(first_hotel_id, i)
    second_in_one = create_condition(second_hotel_id, j)
    
    second_in_zero = create_condition(second_hotel_id, j)
    first_in_one = create_condition(first_hotel_id, i)
    
    first_perm_list = []
    second_perm_list = []
    neither_perm_list = []

    for rank in all_ranks:
        # This logic could be better but I dont really care.
        if first_in_zero(rank) and second_in_one(rank):
            first_perm_list.append(rank)
        elif second_in_zero(rank) and first_in_one(rank):
            second_perm_list.append(rank)
        else:
            neither_perm_list.append(rank)

    return first_perm_list, second_perm_list, neither_perm_list


In [4]:
# This function will print the number of occurences for the combo in pos 0
import itertools

def count_perm_combos(rank_data, target_words, pos0, min = 50):
    word_pair_combinations = list(itertools.combinations(target_words, 2))

    for word1, word2 in word_pair_combinations:
        first, second, _ = filter_perm_in_pos(rank_data, word1, word2, pos0)
        if len(first) < 50 or len(second) < 50:
            continue
        print(word1, word2, len(first), len(second))


### Loading the relevant data.

In [5]:
London3000 = lm.order_set('', 'Bristol1001', 0) # lazy because it has the load in it.

Loading Previously Generated Ranks.


### The Experiment. 

We will first identifiy the